In [17]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.kyobobook.co.kr'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

session = requests.Session()

try:
    response = session.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 예: 페이지 제목 추출
        title = soup.title.string if soup.title else "Title not found"
        print("Page Title:", title)
        
        # 예: 모든 링크 추출
        links = soup.find_all('a')
        print(f"Number of links found: {len(links)}")
        for link in links[:5]:  # 처음 5개 링크만 출력
            print(link.get('href'))
        
        # 여기에 원하는 다른 정보를 추출하는 코드를 추가할 수 있습니다.
        
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)
except requests.exceptions.RequestException as e:
    print("An error occurred:", e)

Page Title: 교보문고 | 대한민국 최고의 도서쇼핑몰
Number of links found: 3122
#contents
https://event.kyobobook.co.kr/detail/222419
https://www.kyobobook.co.kr/
https://ebook.kyobobook.co.kr/
https://sam.kyobobook.co.kr/


In [30]:
# 2. 첫번째 레이어 정보 추출
from bs4 import BeautifulSoup as bs
import re

# 여백 제거
def no_space(text):
    # &nbsp; 제거 (HTML 공백 문자)
    text = re.sub('&nbsp;', ' ', text)
    
    # 연속된 공백 문자를 하나의 공백으로 대체
    text = re.sub(r'\s+', ' ', text)
    
    # 문자열 앞뒤의 공백 제거
    text = text.strip()
    
    return text

# 테스트
# text = "   text   with   &nbsp;   multiple    spaces   "
# print(no_space(text))

In [34]:
soup = bs(text, 'html.parser')
result = soup.find_all('div', {'class' : 'tab_list_wrap'})
#for item in result:
#    print(item)
# 강의에선 class='tab_list' 실제 사이트에선 tab_item
result2 = result[2].find_all('li', {'class' : 'tab_item'})
for item in result2:
    item2 = item.get_text()
    output = re.sub(r'<.*?>','',item2)
    output = no_space(output)
    print(output)

국내도서
서양도서
일본도서
교보Only


In [39]:
# 두번째 레이어 정보 추출
# <div class="category_view_area">로 정의되는 목록 추출
# <div class="folder_box_header">로 구분되는 두번째 레이어의 아이템 추출
result = soup.find_all('div', {'class' : 'category_view_area'})
for item in result:
    #print(item)
    temp1 = item.find_all('div', {'class' : 'fold_box_header'})
    #print(temp1)
    for item2 in temp1:
        temp2 = item2.get_text()
        temp_list = []
        output = re.sub(r'<.*?>','',temp2)
        output = re.sub('컨텐츠 열기', '', output)
        output = no_space(output)
        #print("[", output, "]")
        temp_list.append(output)
        print(temp_list)

['소설']
['시/에세이']
['인문']
['가정/육아']
['요리']
['건강']
['취미/실용/스포츠']
['경제/경영']
['자기계발']
['정치/사회']
['역사/문화']
['종교']
['예술/대중문화']
['중/고등참고서']
['기술/공학']
['외국어']
['과학']
['취업/수험서']
['여행']
['컴퓨터/IT']
['잡지']
['청소년']
['초등참고서']
['유아(0~7세)']
['어린이(초등)']
['만화']
['대학교재']
['한국소개도서']
['교보오리지널']
['문학']
['취미/실용/여행']
['생활/요리/건강']
['예술/건축']
['인문/사회']
['경제/경영']
['과학/기술']
['어린이ELT']
['유아/아동/청소년']
['한국관련도서']
['문구/멀티/비도서']
['ELT/수험서']
['프랑스도서']
['독일도서']
['스페인도서']
['기타도서']
['교재']
['잡지']
['엔터테인먼트']
['만화/애니']
['문학']
['라이트노벨']
['문고']
['신서']
['아동']
['실용서/예술']
['인문/사회']
['자연/기술과학']
['어학/학습']
['문구/멀티/기타']
['중국도서']
['강연']
['여행']
['교보문고 굿즈']
['시그니처 향']
['교보문고 기프트카드']


In [56]:
# 세번째 레이어 정보 추출
# <li class="fold_box">
# <div class="fold_box_header">
# <li class="category_item">
result = soup.find_all('li', {'class' : 'fold_box'})

for item in result:
    temp1 = item.find('div', {'class': 'fold_box_header'})  # find_all() 대신 find() 사용
    temp2 = temp1.get_text()
    output = re.sub('컨텐츠 열기', '', temp2)
    output = no_space(output)
    #print(output)
    
    temp_list = []
    temp3 = item.find_all('li', {'class' : 'category_item'})
    
    for item2 in temp3:
        temp4 = item2.get_text()
        temp4 = no_space(temp4)
        temp_list.append(temp4)
        
    print(output)
    print(temp_list)

소설
['한국소설', '영미소설', '일본소설', '중국소설', '러시아소설', '프랑스소설', '독일소설', '북유럽소설', '그외유럽소설', '기타나라소설', '고전소설/문학선', '세계문학전집', '라이트노벨', '장르소설']
시/에세이
['한국시', '해외시', '테마에세이', '나라별 에세이', '인물/자전적에세이', '청소년 시/에세이', '시/에세이문고']
인문
['인문학일반', '심리학', '교육학', '유아교육', '특수교육', '임용고시', '철학', '문학이론', '한국문학론', '영미문학론', '중국문학론', '세계문학론', '언어학', '독서/글쓰기', '문헌정보학', '역학/사주', '대학교재', '인문교양총서', '인문고전총서', '방송통신대교재']
가정/육아
['결혼/가정', '임신/출산', '육아', '자녀교육', '살림의지혜', '홈인테리어']
요리
['요리일반', '요리에세이', '테마별요리', '베이킹/간식', '계절요리', '재료별요리', '나라별요리', '생활요리', '전문요리', '와인/커피/음료', '요리수험서']
건강
['건강일반', '뇌건강', '한방치료', '자연건강', '건강식사', '질병치료/예방', '다이어트', '운동/트레이닝', '피부관리/메이크업', '건강문고']
취미/실용/스포츠
['가정원예', '홈인테리어/수납', '생활공예/DIY', '살림의지혜', '반려동물', '등산/낚시', '바둑/골프', '무술', '스포츠', '레크레이션/게임', '퀴즈/퍼즐/스도쿠', '무용', '체육', '취미일반', '취미관련상품', '대학교재']
경제/경영
['경영일반', '경영이론', '경영관리', '경영전략', '경제일반', '경제이론', '경제실무', '각국경제', '세무/회계', '마케팅/광고/고객', '유통/창업', '재테크/금융', '무역/운송', '관광/호텔', '경제/경영문고', '대학교재']
자기계발
['성공/처세', '자기능력계발', '비즈니스능력계발', '인간관계', '화술/협상', '청소년자기